In [4]:
import os
import re
import os.path as op
import pandas as pd
from collections import defaultdict
import logging
from typing import Optional, Union

from bids import BIDSLayout
import numpy as np
from pathlib import Path

from pandas import read_csv
from nibabel import loadsave
from bids.layout import parse_file_entities
from bids.layout.writing import build_path

In [2]:
def get_func_filenames_bids(
    paths_to_func_dir: str,
    task_filter: Optional[list] = None,
    ses_filter: Optional[list] = None,
    run_filter: Optional[list] = None,
) -> tuple[list[list[str]], list[float]]:
    """Return the BIDS functional imaging files matching the specified task and session
    filters as well as the first (if multiple) unique repetition time (TR).
    Parameters
    ----------
    paths_to_func_dir : str
        Path to the BIDS (usually derivatives) directory
    task_filter : list, optional
        List of task name(s) to consider, by default `None`
    ses_filter : list, optional
        List of session name(s) to consider, by default `None`
    run_filter : list, optional
        List of run(s) to consider, by default `None`
    Returns
    -------
    tuple[list[list[str]], list[float]]
        Returns two lists with: a list of sorted filenames and a list of TRs.
    """
    logging.debug("Using BIDS to find functional files...")

    layout = BIDSLayout(
        paths_to_func_dir,
        validate=False,
    )

    all_derivatives = layout.get(
        scope="all",
        return_type="file",
        extension=["nii.gz", "gz"],
        suffix="pet",
        task=task_filter or [],
        session=ses_filter or [],
        run=run_filter or [],
    )

    if not all_derivatives:
        raise ValueError(
            f"No functional derivatives were found under {paths_to_func_dir} with the following filters:"
            f"\nExtension: ['nii.gz', 'gz']"
            f"\nSuffix: bold"
            f"\nTask: {task_filter or []}"
            f"\nSession: {ses_filter or []}"
            f"\nRun: {run_filter or []}"
        )

    affines = []
    for file in all_derivatives:
        affines.append(loadsave.load(file).affine)

    similar_fov_dict = separate_by_similar_values(
        all_derivatives, np.array(affines)[:, 0, 0]
    )
    if len(similar_fov_dict) > 1:
        logging.warning(
            f"{len(similar_fov_dict)} different FoV found ! "
            "Files with similar FoV will be computed together. "
            "Computation time may increase."
        )

    separated_files = []
    separated_trs = []
    for file_group in similar_fov_dict.values():
        t_rs = []
        for file in file_group:
            t_rs.append(layout.get_metadata(file)["RepetitionTime"])

        similar_tr_dict = separate_by_similar_values(file_group, t_rs)
        separated_files += list(similar_tr_dict.values())
        separated_trs += list(similar_tr_dict.keys())

        if len(similar_tr_dict) > 1:
            logging.warning(
                "Multiple TR values found ! "
                "Files with similar TR will be computed together. "
                "Computation time may increase."
            )

    return separated_files, separated_trs

In [20]:
bids_dir = Path('/Users/martinnorgaard/Documents/GitHub/petprep_hmc/data')

layout = BIDSLayout(bids_dir, derivatives=True)

In [23]:
layout.derivatives

{'petprep_hmc': BIDS Layout: ...c/data/derivatives/petprep_hmc | Subjects: 2 | Sessions: 3 | Runs: 2}

In [31]:
all_files = layout.get(subject='01', return_type='filename')

In [32]:
all_files

['/Users/martinnorgaard/Documents/GitHub/petprep_hmc/data/derivatives/petprep_hmc/sub-01/ses-baseline/sub-01_ses-baseline_desc-confounds_timeseries.tsv',
 '/Users/martinnorgaard/Documents/GitHub/petprep_hmc/data/derivatives/petprep_hmc/sub-01/ses-baseline/sub-01_ses-baseline_desc-mc_pet.json',
 '/Users/martinnorgaard/Documents/GitHub/petprep_hmc/data/derivatives/petprep_hmc/sub-01/ses-baseline/sub-01_ses-baseline_desc-mc_pet.nii.gz',
 '/Users/martinnorgaard/Documents/GitHub/petprep_hmc/data/derivatives/petprep_hmc/sub-01/ses-baseline/sub-01_ses-baseline_desc-with_motion_correction.gif',
 '/Users/martinnorgaard/Documents/GitHub/petprep_hmc/data/derivatives/petprep_hmc/sub-01/ses-baseline/sub-01_ses-baseline_desc-without_motion_correction.gif',
 '/Users/martinnorgaard/Documents/GitHub/petprep_hmc/data/derivatives/petprep_hmc/sub-01/ses-baseline/sub-01_ses-baseline_movement.png',
 '/Users/martinnorgaard/Documents/GitHub/petprep_hmc/data/derivatives/petprep_hmc/sub-01/ses-baseline/sub-01_s

In [14]:
BIDSLayout?

Init signature:
BIDSLayout(
    root=None,
    validate=True,
    absolute_paths=True,
    derivatives=False,
    config=None,
    sources=None,
    regex_search=False,
    database_path=None,
    reset_database=False,
    indexer=None,
    **indexer_kwargs,
)
Docstring:     
Layout class representing an entire BIDS dataset.

Parameters
----------
root : str
    The root directory of the BIDS dataset.
validate : bool, optional
    If True, all files are checked for BIDS compliance when first indexed,
    and non-compliant files are ignored. This provides a convenient way to
    restrict file indexing to only those files defined in the "core" BIDS
    spec, as setting validate=True will lead files in supplementary folders
    like derivatives/, code/, etc. to be ignored.
absolute_paths : bool, optional
    If True, queries always return absolute paths.
    If False, queries return relative paths (for files and
    directories).
derivatives : bool or str or list, optional
    Specifies w